### Trained the GAN for 1300 epochs

4 features : PRECT, QBP, PS, TBP


In [0]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.special as s
import pickle
import os
import random
import torch
import seaborn as sns
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

In [0]:
# TODO: Enter the path to the stored pickle file
d_vars = pickle.load(open('data.pkl','rb'))

In [0]:
# We will only use non-zero values of PRECT to train our network
d_non_zero = d_vars[d_vars.PRECT>0]

In [0]:
d_non_zero.shape

In [0]:
d_non_zero.describe()

#### Feature Transformation

In [0]:
d_non_zero['PRECT_NEW'] = s.boxcox(d_non_zero.PRECT, 0.1)
d_non_zero['QBP_NEW'] = d_non_zero.QBP**(1/4) #s.boxcox(d_non_zero.QBP, 0.4)
d_non_zero['SHFLX_NEW'] = d_non_zero.SHFLX*(10**(-2))
d_non_zero['LHFLX_NEW'] = d_non_zero.LHFLX*(10**(-2))
d_non_zero['PS_NEW'] = d_non_zero.PS*(10**(-4))

In [0]:
d_non_zero = (d_non_zero).astype({'QBP_NEW': 'float32','LHFLX_NEW': 'float32','SHFLX_NEW': 'float32','PS_NEW': 'float32'})

In [0]:
d_non_zero.dtypes

###### Plot the transformed features

In [0]:
plt.figure(figsize=(20,15))
vars_to_plot = ['PRECT_NEW', 'QBP_NEW', 'PS_NEW']
for i, var in enumerate(vars_to_plot):
  plt.subplot(3,3,i+1)
  plt.hist(d_non_zero[var], density=True, bins=20)
  plt.xlabel('Transformed '+var)
  plt.ylabel('Density')

In [0]:
# TODO: Select which features to train the GAN on
# (keep in mind what features you select should effect the architecture of the model - ideally for more features you will need a deeper model)
data=np.array(d_non_zero[['PRECT_NEW','QBP_NEW','PS_NEW','TBP']])
data

In [0]:
data.shape

### GAN Model

With noisy 0 to 0.2 range of labels for real data, 0.8 to 1 range of labels for fake data

In [0]:
# TODO: Set the batch size
data_loader = torch.utils.data.DataLoader(data, batch_size=1024, shuffle=False)

# Num batches
num_batches = len(data_loader)
num_batches

###### Define Discriminator Network

In [0]:
# TODO: Modify the architecture if needed
class DiscriminatorNet(torch.nn.Module):
    def __init__(self):
        super(DiscriminatorNet, self).__init__()
        # TODO: modify n_features if more variables selected
        n_features = 4
        n_out = 1
        
        self.hidden0 = nn.Sequential( 
            nn.Linear(n_features, 1024),
            nn.LeakyReLU(0.2),
  
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            # nn.Dropout(0.3)
        )
        self.hidden3 = nn.Sequential(
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.out = nn.Sequential(
            torch.nn.Linear(128, n_out),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.hidden0(x.cuda())
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        x = self.out(x)
        return x
discriminator = DiscriminatorNet()

###### Define Generator Network

In [0]:
# TODO: Modify the architecture if needed
class GeneratorNet(torch.nn.Module):
    def __init__(self):
        super(GeneratorNet, self).__init__()
        n_features = 100
        # TODO: modify n_out if more variables selected
        n_out = 4
        
        self.hidden0 = nn.Sequential( 
            nn.Linear(n_features, 256),
            nn.ReLU(),
  
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 1024),
            nn.ReLU(),
            # nn.Dropout(0.3)
        )
        self.hidden3 = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.out = nn.Sequential(
            torch.nn.Linear(256, n_out),
        )


    def forward(self, x):
        x = self.hidden0(x.cuda())
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        
        x = self.out(x)
        return x
generator = GeneratorNet()

In [0]:
use_cuda=True
if use_cuda and torch.cuda.is_available():
    discriminator.cuda()
    generator.cuda()

In [0]:
def noise(size):
    '''
    Generates a 1-d vector of gaussian sampled random values
    '''
    n = Variable(torch.randn(size, 100))
    n.cuda()
    return n

In [0]:
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0004)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0001)

In [0]:
loss = nn.BCELoss()

In [0]:
torch.cuda.is_available()

In [0]:
def train_discriminator(optimizer, data, labels):

    N = real_data.size(0)

    # Reset gradients
    optimizer.zero_grad()
    
    # 1.1 Train on Real Data
    prediction = discriminator(data)
    prediction.cuda()
    labels = labels.cuda()
    
    error_total = loss(prediction, labels)
    error_total.backward()

    optimizer.step()
    
    # Return error and predictions for real and fake inputs
    return error_total, prediction

In [0]:
def train_generator(optimizer, fake_data, labels):
    N = fake_data.size(0)
    # Reset gradients
    optimizer.zero_grad()

    # Sample noise and generate fake data
    prediction = discriminator(fake_data)
    prediction.cuda()

    labels = labels.cuda()
    
    # Calculate error and backpropagate
    error = loss(prediction, labels)

    error.backward()
    # Update weights with gradients
    optimizer.step()
    # Return error
    return error

### Checkpoints

Add a path to a checkpoint folder to store training checkpoints

In [0]:
# TODO: Add correct path
drive_root = "/content/drive/My Drive/Capstone Project/GAN-Capstone/"
checkpoint_dir = os.path.join(drive_root, "checkpoints")
# your name here
checkpoint_dir = os.path.join(checkpoint_dir, "akshata")

In [0]:
## TODO: if you are starting training from scratch, make sure you rm -r the contents of the checkpoint directory

In [0]:
print("Checkpoints directory is", checkpoint_dir)
if os.path.exists(checkpoint_dir):
  print("Checkpoints folder already exists")
else:
  print("Creating a checkpoints directory")
  os.makedirs(checkpoint_dir)

In [0]:
# Restore the latest checkpoints
chkpts = os.listdir(checkpoint_dir)
if chkpts:
  latest = chkpts[0]
  print(latest)
  saved = torch.load(checkpoint_dir+'/'+latest)
  generator.load_state_dict(saved['gen_state_dict'])
  discriminator.load_state_dict(saved['disc_state_dict'])
  g_optimizer.load_state_dict(saved['gen_optimizer_state_dict'])
  d_optimizer.load_state_dict(saved['disc_optimizer_state_dict'])
  current_epoch = saved['epoch']+1
  g_error = saved['gen_loss']
  d_error = saved['disc_loss']
else:
  current_epoch = 0

current_epoch

In [0]:
dtype = torch.cuda.FloatTensor

In [0]:
# TODO: Set the number of epochs
num_epochs = 1300

for epoch in range(current_epoch, num_epochs):
  
  # TODO: modify the columns of the result dataframe if more variables selected
  results = pd.DataFrame(columns=['Prect','QBP','PS','TBP'])

  for n_batch,real_batch in enumerate(data_loader):
    N = real_batch.size(0)
   
    # 1. Train Discriminator
    real_data = real_batch
    real_data.cuda()
    
    # Generate fake data and detach 
    # (so gradients are not calculated for generator)
    noi = noise(N)
    noi.cuda()

    fake_data = generator(noi).detach()
    fake_data.cuda()

    ## add one percent noise to the labels
    one_prec_N = int((int(0.1*N)+1)/2)

    ## for real data - randomly shuffled labels in range from 0 to 0.2
    y_dis0 = np.zeros(N)
    y_dis0[:N-one_prec_N] = np.random.uniform(0.0,0.2000000001,N-one_prec_N)
    y_dis0[N-one_prec_N:] = np.random.uniform(0.8,1.0000000001,one_prec_N)
    random.shuffle(y_dis0)

    ## for fake data - randomly shuffled labels in range from 0.8 to 1.0
    y_dis1 = np.ones(N)
    y_dis1[:N-one_prec_N]= np.random.uniform(0.8,1.0000000001,N-one_prec_N)
    y_dis1[N-one_prec_N:]= np.random.uniform(0.0,0.2000000001,one_prec_N)
    random.shuffle(y_dis1)

    y_dis = np.concatenate((y_dis0,y_dis1),axis=0)
    y_torch = Variable(torch.from_numpy(y_dis).float())
    y_torch.cuda()
    
    #Train D
    error_real, pred_real = train_discriminator(d_optimizer, real_data ,Variable(torch.from_numpy(y_dis0).float()))
    error_fake, pred_fake = train_discriminator(d_optimizer, fake_data ,Variable(torch.from_numpy(y_dis1).float()))

    
    d_error = error_real + error_fake

    d_pred = []
    d_pred.append(pred_real)
    d_pred.append(pred_fake)    

    # 2. Train Generator
    fake_data = generator(noise(N))
    fake_data.cuda()
    
    # Train G
    y_gen = np.zeros(N)
    y_torch_gen = Variable(torch.from_numpy(y_gen).float())
    y_torch_gen.cuda()

    g_error = train_generator(g_optimizer, fake_data, y_torch_gen)

    # print d error and g error
    if (n_batch) % 100 == 0: 
      """logger.display_status(
                  epoch, num_epochs, n_batch, num_batches,
                  d_error, g_error
              )"""
      print(d_error,g_error,fake_data[0],epoch)
    
    results = results.append(pd.DataFrame(fake_data.tolist(), columns=results.columns))
  
  # save checkpoint
  checkpoint_path = os.path.join(checkpoint_dir, 'checkpoint.pt')
  torch.save({'epoch': epoch,
              'gen_state_dict': generator.state_dict(),
              'disc_state_dict': discriminator.state_dict(),
              'gen_optimizer_state_dict': g_optimizer.state_dict(),
              'disc_optimizer_state_dict': d_optimizer.state_dict(),
              'gen_loss': g_error,
              'disc_loss': d_error,},
             checkpoint_path)
  
  ## TODO:  specify your results pickle file here
  pickle.dump(results, open('results_akshata_new.pkl', 'wb'))
  print('Saved model at ', checkpoint_path)

##### Explore the results

In [0]:
results = pickle.load(open('results_akshata_new.pkl', 'rb'))

In [0]:
results.describe()

Inverse transform the data to get values in the correct range

In [0]:
q = results.QBP**4
p = s.inv_boxcox(results.Prect, 0.1)
t = results.TBP
ps = (results.PS) * (10**4)

In [0]:
ans = pd.concat([p.reset_index(drop=True),
                 q.reset_index(drop=True),
                 t.reset_index(drop=True),
                 ps.reset_index(drop=True),
                 ], axis=1)

In [0]:
ans.describe()

A scatter plot of PRECT vs QBP

In [0]:
plt.scatter(ans.QBP, ans.Prect, alpha = 0.2)
plt.xlabel('QBP')
plt.ylabel('PRECT')
plt.show()

A scatter plot of PRECT vs TBP

In [0]:
plt.scatter(ans.TBP, ans.Prect, alpha=0.2)
plt.xlabel('TBP')
plt.ylabel('PRECT')
plt.show()

Distribution of all generated variables

In [0]:
plt.figure(figsize=(20,15))
vars_to_plot = results.columns.to_list()
for i, var in enumerate(vars_to_plot):
  plt.subplot(3,3,i+1)
  plt.hist(results[var], density=True, bins=20)
  plt.xlabel('Predicted '+var)
  plt.ylabel('Density')

Plot variance of PRECT vs Bins of QBP

In [0]:
ans_sorted = ans.sort_values(by='QBP')

In [0]:
prect_vars_result = []
qbp_bins_result_str = []
for split in np.array_split(ans_sorted,10,axis = 0):
  prect_vars_result.append(np.std(split.Prect)**2)
  qbp_bins_result_str.append("{:.3f}".format(split.QBP.min()*(10**3))+" - "+"{:.3f}".format(split.QBP.max()*(10**3)))

In [0]:
plt.plot(prect_vars_result)
plt.xticks(list(range(0,10)),labels=qbp_bins_result_str,rotation=90)
plt.xlabel('QBP Range (1e-3)')
plt.ylabel('PRECT Variance')
plt.show()